In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import CRUD module
from animalshelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Username and password for CRUD module
username = "aacuser"
password = "aacuser_password"

# Connect to database via CRUD Module
db = AnimalShelter()

# Populate data table with all records from AnimalShelter
df = pd.DataFrame.from_records(db.find({}))

# Drop '_id' column - contains invalid 'ObjectID' type which causes data_table to crash
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load Grazioso Salvare logo
image_filename = 'GraziosoSalvareLogo1.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Dash layout
app.layout = html.Div([
    # Logo with link to Grazioso Salvare homepage
    html.A(
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
        href='https://www.snhu.edu'),
    
    # Title and unique identifier
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Center(html.H2('David France - CS-340')),
    html.Hr(),
    
    # Rescue animal radio buttons with reset button
    html.Center(html.Div(
        [html.H4('Rescue Type'),
        dcc.RadioItems([{'label':'Water','value':'water'},
                    {'label':'Mountain/Wilderness', 'value':'mtw'}, 
                    {'label':'Disaster/Individual Tracking', 'value':'dit'}],
                       value = False,
                  id='radio_btn'),
        html.Br(),
        html.Button(id='reset_btn', n_clicks=0, children='Reset')]
    )),
    html.Hr(),
    
    # Data table displaying information for all or selected animals
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable = 'single', # Allow selection of one row at a time
        selected_rows = [0],       # Default select first row
        page_size = 10,            # Set results per page to 10
        page_current = 0,          # Set first page as default
        sort_action = 'native'     # Allow sorting of columns
    ),
    html.Br(),
    html.Hr(),
    
    # Set up areas for donut chart of breeds on the lower left and 
    # geolocation map on lower right of dashboard
    html.Div(className='row',
        style={'display' : 'flex'},
        children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################

# Radio button selection callback to handle rescue animal queries  
@app.callback(
            Output('datatable-id','data'),
            Output('datatable-id','selected_rows'),
            Output('datatable-id','page_current'),
            Input('radio_btn', 'value')
            )
def update_dashboard(filter_type):
    df = pd.DataFrame.from_records(db.find({}))

    # Water type constraints
    if filter_type == 'water':
        df = pd.DataFrame.from_records(db.find({'animal_type': 'Dog', 
                                                'breed': {'$in': ['Chesa Bay Retr Mix',   
                                                                  'Labrador Retriever Mix',
                                                                  'Newfoundland']},
                                                'sex_upon_outcome': 'Intact Female',
                                                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        })) 
        
    # Mountain or water type constraints    
    if filter_type == 'mtw':
        df = pd.DataFrame.from_records(db.find({'animal_type': 'Dog', 
                                                'breed': {'$in': ['German Shepherd', 
                                                                  'Alaskan Malamute', 
                                                                  'Old English Sheepdog',
                                                                  'Siberian Husky',
                                                                  'Rottweiler']},
                                                'sex_upon_outcome': 'Intact Male',
                                                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        })) 

    # Disaster or individual tracking constraints
    if filter_type == 'dit':
        df = pd.DataFrame.from_records(db.find({'animal_type': 'Dog', 
                                                'breed': {'$in': ['Doberman Pinsch', 
                                                                  'German Shepherd', 
                                                                  'Golden Retriever',
                                                                  'Bloodhound',
                                                                  'Rottweiler']},
                                                'sex_upon_outcome': 'Intact Male',
                                                'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        })) 
        
    df.drop(columns=['_id'],inplace=True)

    return df.to_dict('records'), [0], 0


# Reset button callback unselects all radio buttons
@app.callback(
            Output('radio_btn','value'),
            Input('reset_btn', 'n_clicks')
            )
def update_dashboard(n_clicks):
    if n_clicks > 0:
        return False

    
# Donut chart to display the breeds of animal based on quantity represented in
# the data table
@app.callback(
        Output('graph-id', 'children'),
        Input('datatable-id', "derived_virtual_data")
        )
def update_graphs(viewData):
    pass
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    fig = px.pie(dff, names='breed', 
                 title='Preferred Animals', 
                 color_discrete_sequence=px.colors.qualitative.Prism,
                 hole=0.4)
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    fig.update_layout(title_x=0.5, title_font_size = 25)

    return dcc.Graph(figure=fig)
    
    
# Highlight a row on the data table when the user selects it
@app.callback(
        Output('datatable-id', 'style_data_conditional'),
        Input('datatable-id', 'selected_rows')
        )
def update_styles(selected_columns):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Update marker on map to reflect row selected
@app.callback(
        Output('map-id', "children"),
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows")
        )
def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
     # Because we only allow single row selection, the list can 
     # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
    
    # Map is centered on Austin TX at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=8, children=[
           dl.TileLayer(id="base-layer-id"),
               
           # Marker with tool tip and popup
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], # Column 13 and 14 define grid coordinates
              children=[
              dl.Tooltip(dff.iloc[row,4]), # Col 4 is breed of animal
              dl.Popup([
                 html.H2("Animal Name"),
                 html.H3(dff.iloc[row,9])  # Col 9 is name of animal
             ])
          ])
       ])
    ]


app.run_server(debug=True)


C:\Users\david\AppData\Local\Programs\Python\Python311\Lib\site-packages\dash\dash.py:525: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
